# Default mcKinnon settings, Except:
- a bit different region
- ERA-i data 
- 1979 - 2017 (instead of 1982 - 2015)
- data on 2.5 degrees

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Nov 13 14:40:40 2018

@author: semvijverberg
"""
import os, sys
os.chdir('/Users/semvijverberg/surfdrive/Scripts/Extracting_precursor/')
script_dir = os.getcwd()
if sys.version[:1] == '3':
    from importlib import reload as rel
import func_mcK
from ROC_score import ROC_score_wrapper
import numpy as np
import xarray as xr 
import pandas as pd
import cartopy.crs as ccrs
xrplot = func_mcK.xarray_plot
import matplotlib.pyplot as plt
import scipy

base_path = "/Users/semvijverberg/surfdrive/Data_ERAint/"
exp_folder = ''
path_raw = os.path.join(base_path, 'input_raw')
path_pp  = os.path.join(base_path, 'input_pp')
if os.path.isdir(path_raw) == False : os.makedirs(path_raw) 
if os.path.isdir(path_pp) == False: os.makedirs(path_pp)



#ex_dic_path = "T95_sst_NOAA_default_settings.npy"
ex_dic_path = "ERA_T2mmax_sst_default_settings.npy"
ex = np.load(ex_dic_path, encoding='latin1').item()

ex['figpathbase'] = os.path.join(ex['figpathbase'], '{}_{}'.format(
        ex['RV_name'], ex['name']))
if os.path.isdir(ex['figpathbase']) == False: os.makedirs(ex['figpathbase'])

#'Mckinnonplot', 'U.S.', 'U.S.cluster', 'PEPrectangle', 'Pacific', 'Whole', 'Northern', 'Southern'
def oneyr(datetime):
    return datetime.where(datetime.year==datetime.year[0]).dropna()


ex['region'] = 'Whole'
print(ex['region'])

if ex['load_mcK'] == True:
    # Load in mckinnon Time series
    T95name = 'PEP-T95TimeSeries.txt'
    RVtsfull, datesmcK = func_mcK.read_T95(T95name, ex)
    datesRV = func_mcK.make_datestr(datesmcK, ex)
else:
    # load ERA-i Time series
    ex['sstartdate'] = ex['sstartdate'].replace(ex['sstartdate'][:4], 
      str(ex['startyear']) )
    ex['senddate'] = ex['senddate'].replace(ex['senddate'][:4], 
      str(ex['startyear']) )
    print('\nimportRV_1dts is true, so the 1D time serie given with name {}\n'
              '{} is imported.'.format(ex['RV_name'],ex['RVts_filename']))
    filename = os.path.join(ex['RV1d_ts_path'], ex['RVts_filename'])
    dicRV = np.load(filename,  encoding='latin1').item()
    RVtsfull = dicRV['RVfullts']
    RVhour   = RVtsfull.time[0].dt.hour.values
    datesRV = func_mcK.make_datestr(pd.to_datetime(RVtsfull.time.values), ex)
    # add RVhour to daily dates
    datesRV = datesRV + pd.Timedelta(int(RVhour), unit='h')

# Selected Time series of T95 ex['sstartdate'] until ex['senddate']
RVts = RVtsfull.sel(time=datesRV)

# Load in external ncdf
#filename = '{}_1979-2017_2mar_31aug_dt-1days_2.5deg.nc'.format(ex['name'])
filename = '{}_{}-{}_2jan_31aug_dt-1days_2.5deg.nc'.format(ex['name'],
            ex['startyear'], ex['endyear'])
# full globe - full time series
varfullgl = func_mcK.import_array(filename, ex)

# filter out outliers of sst
if ex['name']=='sst':
    varfullgl.where(varfullgl.values < 3.5*varfullgl.std().values)



RV_ts, datesmcK = func_mcK.time_mean_bins(RVts, ex)
expanded_time = func_mcK.expand_times_for_lags(datesmcK, ex)
# Converting Mckinnon timestemp to match xarray timestemp
expandeddaysmcK = func_mcK.to_datesmcK(expanded_time, expanded_time[0].hour, varfullgl.time[0].dt.hour)
# region mckinnon - expanded time series
Prec_reg = func_mcK.find_region(varfullgl.sel(time=expandeddaysmcK), region=ex['region'])[0]
Prec_reg, datesvar = func_mcK.time_mean_bins(Prec_reg, ex)

# binary time serie when T95 exceeds 1 std
ex['hotdaythres'] = RV_ts.mean(dim='time').values + RV_ts.std().values
#ex['hotdaythres'] = np.percentile(RV_ts.values, 80)
# If method == 'random' - Run until ROC has converged


## plotting same figure as in paper
func_mcK.plot_oneyr_events(RV_ts, ex['hotdaythres'], 2012)


print_ex = ['RV_name', 'name', 'grid_res', 'startyear', 'endyear', 
            'sstartdate', 'senddate', 'n_conv', 'leave_n_out',
            'method', 'ROC_leave_n_out', 'wghts_std_anom', 
            'wghts_accross_lags', 'splittrainfeat', 'n_strongest',
            'n_std', 'tfreq', 'lags', 'n_yrs', 'hotdaythres']

max_key_len = max([len(i) for i in print_ex])
for key in print_ex:
    key_len = len(key)
    expand = max_key_len - key_len
    key_exp = key + ' ' * expand
    print('\'{}\'\t\t{}'.format(key_exp, ex[key]))

def main(RV_ts, Prec_reg, ex):
    ex['score_per_run'] = []
    if ex['leave_n_out'] == False : ex['n_conv'] = 1
    # Purely train-test based on iterating over all years:
    lats = Prec_reg.latitude
    lons = Prec_reg.longitude
    array = np.zeros( (ex['n_conv'], len(ex['lags']), len(lats), len(lons)) )
    patterns = xr.DataArray(data=array, coords=[range(ex['n_conv']), ex['lags'], lats, lons], 
                          dims=['n_tests', 'lag','latitude','longitude'], 
                          name='{}_tests_patterns'.format(ex['n_conv']), attrs={'units':'Kelvin'})
    
    for n in range(ex['n_conv']):
        ex['n'] = n

        # do single run
        # =============================================================================
        # Calculate Precursor
        # =============================================================================
        test, train, ds_mcK, ds_Sem, ex = func_mcK.find_precursor(RV_ts, Prec_reg, ex)
    
        # =============================================================================
        # Calculate ROC score
        # =============================================================================
        ex = ROC_score_wrapper(test, train, ds_mcK, ds_Sem, ex)
        l_ds_Sem       = [ex['score_per_run'][i][3] for i in range(len(ex['score_per_run']))]
        patterns[n,:,:,:] = l_ds_Sem[n]['pattern']
        
        
        ex['n'] += 1
        if ex['leave_n_out'] == False:
            # only need single run
            break
    return ex, patterns

        
ex, patterns = main(RV_ts, Prec_reg, ex)

Whole

importRV_1dts is true, so the 1D time serie given with name t2mmax
t2mmax_1979-2017_averAggljacc_tf14_n8__to_t2mmax.npy is imported.
'RV_name           '		t2mmax
'name              '		sst
'grid_res          '		2.5
'startyear         '		1979
'endyear           '		2017
'sstartdate        '		1979-06-24
'senddate          '		1979-08-22
'n_conv            '		39
'leave_n_out       '		True
'method            '		iter
'ROC_leave_n_out   '		False
'wghts_std_anom    '		False
'wghts_accross_lags'		False
'splittrainfeat    '		False
'n_strongest       '		15
'n_std             '		1.5
'tfreq             '		1
'lags              '		[0, 6, 12, 18]
'n_yrs             '		39
'hotdaythres       '		2.903553828333342
test year is [1979], with 3 events
test year is [1980], with 39 events
test year is [1981], with 5 events
test year is [1982], with 2 events
test year is [1983], with 24 events
test year is [1984], with 0 events
test year is [1985], with 3 events
test year is [1986], with 10 events
test yea

# Simple mean over all runs
Taking the mean over all generated patterns, each found from (n_yrs - 1) years

In [1]:
events_per_year = [ex['score_per_run'][i][1] for i in range(len(ex['score_per_run']))]
l_ds_mcK        = [ex['score_per_run'][i][2] for i in range(len(ex['score_per_run']))]
l_ds_Sem        = [ex['score_per_run'][i][3] for i in range(len(ex['score_per_run']))]
ran_ROCS        = [ex['score_per_run'][i][4] for i in range(len(ex['score_per_run']))]
score_mcK       = np.round(ex['score_per_run'][-1][2]['score'], 2)
score_Sem       = np.round(ex['score_per_run'][-1][3]['score'], 2)


# mcKinnon plot
filename = os.path.join(ex['exp_folder'], 'mcKinnon composite_tf{}_{}'.format(
            ex['tfreq'], ex['lags']))
func_mcK.plotting_wrapper(l_ds_mcK[0]['pattern'], filename, ex)

# Sem plot
mean_n_patterns = patterns.mean(dim='n_tests')
mean_n_patterns.attrs['units'] = 'mean over {} runs'.format(ex['n_conv'])
mean_n_patterns.name = 'ROC {}'.format(score_Sem.values)
filename = os.path.join(ex['exp_folder'], 'mean_over_{}_tests'.format(ex['n_conv']) )
func_mcK.plotting_wrapper(mean_n_patterns, filename, ex)

NameError: name 'ex' is not defined

# Counting times gridcells were extracted

In [ ]:
#%% Counting times gridcells were extracted
if ex['leave_n_out']:
    n_lags = patterns.sel(n_tests=0).lag.size
    n_lats = patterns.sel(n_tests=0).latitude.size
    n_lons = patterns.sel(n_tests=0).longitude.size
    
    pers_patt = patterns.sel(n_tests=0).copy()
    arrpatt = np.nan_to_num(patterns.values)
    mask_patt = (arrpatt != 0)
    arrpatt[mask_patt] = 1
    wghts = np.zeros( (n_lags, n_lats, n_lons) )
    #plt.imshow(arrpatt[0,0]) ; plt.colorbar()
    for l in ex['lags']:
        i = ex['lags'].index(l)
        wghts[i] = np.sum(arrpatt[:,i,:,:], axis=0)
    pers_patt.values = wghts
    
    pers_patt.attrs['units'] = 'persistence pattern over {} runs'.format(ex['n_conv'])
    pers_patt.attrs['title'] = 'ROC {}'.format(score_Sem.values)
    filename = os.path.join(ex['exp_folder'], 'mean_over_{}_tests'.format(ex['n_conv']) )
    kwrgs = dict( {'title' : pers_patt.name, 'clevels' : 'notdefault', 'steps':17,
                    'vmin' : pers_patt.min().values, 'vmax' : pers_patt.max().values, 
                   'cmap' : plt.cm.gist_heat_r, 'column' : 2} )
    func_mcK.plotting_wrapper(pers_patt, filename, ex, kwrgs=kwrgs)

# Only keep gridcells that were extracted every run

In [ ]:
#%% Only keep gridcells that were extracted every run
if ex['leave_n_out']:
    pers_patt_filter = patterns.sel(n_tests=0).copy().drop('n_tests')
    ex['persistence_criteria'] = len(set(RV_ts.time.dt.year.values))
    mask_pers = (wghts == ex['persistence_criteria'])
    mean_n_patterns = patterns.mean(dim='n_tests')
    mean_n_patterns.coords['mask'] = (('lag', 'latitude','longitude'), mask_pers)
    #pers_patt_filter = mask_pers * patterns.mean(dim='n_tests')
    pers_patt_filter.attrs['units'] = 'weighted by persistence of pattern over {} runs'.format(ex['n_conv'])
    pers_patt_filter.name = 'ROC {}'.format(score_Sem.values)
    func_mcK.plotting_wrapper(pers_patt_filter, filename, ex, kwrgs=None)